In [ ]:
# !pip install openai pandas scikit-learn

import pandas as pd
import numpy as np
import openai
from openai import AzureOpenAI
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
# 🎯 임베딩용 Azure 클라이언트
embedding_client = AzureOpenAI(
  api_key = "",
  api_version = "",
  azure_endpoint = ""
)


# 🎯 챗용 Azure 클라이언트
chat_client = AzureOpenAI(
  api_key="",
  api_version="",
  azure_endpoint=""
)


In [ ]:
#✅ 2. CSV 로딩 함수

def load_embedded_csv(filepath):
    df = pd.read_csv(filepath)
    df['ada_v2'] = df['ada_v2'].apply(lambda x: eval(x) if isinstance(x, str) else x)
    return df

In [ ]:
#✅ 3. 임베딩 함수
def generate_embeddings(text, model="text-embedding-ada-002"):
    response = embedding_client.embeddings.create(
        input=[text],
        model=model
    )
    return response.data[0].embedding


In [ ]:
#✅ 4. 유사 문서 검색 / name 인식 / 필터를 추가 / 이름 출력(최종)

def filter_df_by_query(query, df):
    # ① 이름 기반 필터 우선 적용
    name_filtered = df[df['name'].str.contains(query, na=False)]

    if not name_filtered.empty:
        print("✅ 이름 기반 필터링 적용:", name_filtered['name'].tolist())
        return name_filtered

    # ② 다중 필드 기반 필터
    multi_filtered = df[
        df['orders'].str.contains(query, na=False) |
        df['movementFamily'].str.contains(query, na=False) |
        df['activities'].str.contains(query, na=False) |
        df['content'].str.contains(query, na=False)
    ]

    if not multi_filtered.empty:
        print("✅ 다중 필드 기반 필터링 적용")
        return multi_filtered

    print("⚠️ 필터링 실패 → 전체 데이터 사용")
    return df


def retrieve_relevant_context(query, df, top_k=3, embed_model="text-embedding-ada-002"):
    # 1. 질문 임베딩 생성
    query_emb = generate_embeddings(query, model=embed_model)

    # 2. 유사도 계산
    df['score'] = df['ada_v2'].apply(lambda x: cosine_similarity([x], [query_emb])[0][0])

    # ✅ 3. 이름 포함된 문서에는 점수 가중치 추가
    df.loc[df['name'].str.contains(query, na=False), 'score'] += 1.0

    # 4. Top-K 문서 선택
    top_docs = df.sort_values(by="score", ascending=False).head(top_k).reset_index()

    context_blocks = []
    citations = []

    print("🔍 유사도 상위 문서 (Top {}):".format(top_k))
    for i, row in top_docs.iterrows():
        ref_id = f"[{i+1}]"
        name = row.get("name", "")
        score = round(row["score"], 4)
        print(f"{ref_id} {name} (score: {score})")

        # GPT에 넘길 문서 블록 구성
        text = f"""{ref_id}
        이름: {name}
        한자: {row.get("nameHanja", "")}
        출생지: {row.get("addressBirth", "")}
        훈격: {row.get("orders", "")}
        독립운동 분야: {row.get("movementFamily", "")}
        주요 활동: {row.get("activities", "")}
        관련 단체: {row.get("engagedOrganizations", "")}
        내용: {row['content']}
        """
        context_blocks.append(text)

        # Citation 구성
        citation_info = {
            "index": i + 1,
            "title": name,
            "reference": row.get("references")
        }
        citations.append(citation_info)

    context = "\n\n".join(context_blocks)
    return context, citations


def rag_chatbot(
    query,
    df,
    top_k=5,
    embed_model="text-embedding-ada-002",
    chat_model="gpt-4o",
    system_prompt="너는 독립운동 전문가야. 문서 내 '이름'과 사용자 질문을 비교해서 정확히 답변해줘."
):
    # 1. 필터링
    filtered_df = filter_df_by_query(query, df)

    if filtered_df.empty:
        print("⚠️ 이름/내용 필터링 실패 → 전체 데이터 사용")
        filtered_df = df

    # 2. 유사도 기반 context 생성
    context, citations = retrieve_relevant_context(query, filtered_df, top_k=top_k, embed_model=embed_model)
    
    # 3. GPT 호출
    response = chat_client.chat.completions.create(
        model=chat_model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"질문: {query}\n\n아래 문서를 참고해서 대답해줘:\n\n{context}"}
        ]
    )

    # 4. 결과 출력
    content_text = response.choices[0].message.content

    print("🤖 GPT 응답:\n")
    print(content_text)

    print("\n📚 Citations:")
    for c in citations:
        print(f"[{c['index']}] {c['title']}\n출처: {c['reference']}\n")

    return content_text, citations


In [ ]:
#✅ 5. 챗 응답 함수
def ask_gpt(query, context, model="gpt-4o",system_prompt="너는 한국 독립운동 전문가야. 아래 문서를 참고해서 정확하게 대답해줘."):
    response = chat_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"질문: {query}\n\n참고 문서:\n{context}"},
            {"role": "user", "content": f"인물: {query}\n\n아래 문서를 참고해 이 인물에 대한 내용을 찾아줘.\n\n{context}"}
        ]
    )
    return response.choices[0].message.content




In [ ]:
# ✅ 사용 예시
response, citations = rag_chatbot(
    query="애국장과 관련있는 독립운동가 3인?",
    df=df,
    embed_model="text-embedding-ada-002",
    chat_model="gpt-4o",
    top_k=5,  # ← top_k를 늘리면 더 넓은 문서 범위를 검색함
    
    system_prompt = """
    너는 한국 독립운동 인물 전문 AI야. 친절하고 생동감있게 이야기하듯 말해줘. 유사도 높은 인물 중심으로 내용을 대답해.
    문서 중 '관련 인물', '활동 내역'에서 연결점을 찾아  
    사용자가 물어본 인물과 연관된 다른 인물이 있다면 이름을 명확히 알려줘.

    - 반드시 아래 제공된 문서들만 참고해서 대답해.  
    - 문서에 없는 정보는 "문서에 해당 내용이 없습니다."라고 분명히 말해줘.  
    - 추론하거나 외부 지식을 추가하지 마.  
    - 사용자의 질문이 인물, 활동, 상훈, 소속 단체, 관련 인물에 대한 것이면 해당 문서 필드를 찾아서 설명해줘.

    연결된 근거가 없으면 '연관된 인물 정보는 문서에 없습니다'라고 답해줘.
    """

)
